In [8]:
import netket as nk
from qiskit.chemistry.drivers import PySCFDriver
from qiskit.chemistry import FermionicOperator
import numpy as np

from system_dicts import *

#systemData = b631g_H2[4]
systemData = sto3g_H2_eq
systemData = sto3g_LiH_eq

#systemData={'driver_string': 'H 0.0 0.0 0.0; H 0.0 0.0 0.734', 'basis': 'sto3g'}; n_electrons=2
#systemData={'driver_string': 'H 0.0 0.0 0.0; H 0.0 0.0 0.734', 'basis': '631g'}; n_electrons=2
#systemData={'driver_string': 'Li 0.0 0.0 0.0; H 0.0 0.0 1.54', 'basis': 'sto3g'}; n_electrons=4
#systemData={'driver_string': 'C 0.0 0.0 0.0; C 0.0 0.0 1.26', 'basis': 'sto3g'}; n_electrons=12

print(systemData)
driver = PySCFDriver(   atom=systemData["atomstring"],
                        basis=systemData["basis"])
                        
mol = driver.run()
OB = mol.one_body_integrals
TB = mol.two_body_integrals

nre = mol.nuclear_repulsion_energy

FerOp = FermionicOperator(OB, TB)
#FerOp.fermion_mode_elimination([0])
mapping = FerOp.mapping('jordan_wigner')
weights = [w[0] for w in mapping.paulis]
operators = [w[1].to_label() for w in mapping.paulis]
ha = nk.operator.PauliStrings(operators, weights)

g = nk.graph.Hypercube(n_dim=1, length=ha.hilbert.size, pbc=False)
hi = nk.hilbert.Qubit(graph=g)
assert(hi.size==ha.hilbert.size)


{'molecule': 'LiH', 'basis': 'sto3g', 'n_basisfuncs': 12, 'distance': 1.5474999999999999, 'atomstring': 'Li 0.0 0.0 0.0; H 0.0 0.0 1.547500', 'eq': True, 'tot_energy': -7.882762239224967, 'nuc_rep_energy': 1.0258685828820677, 'n_electrons': 4}


In [9]:
res = nk.exact.lanczos_ed(ha, first_n=1, compute_eigenvectors=True)
print("Exact ground state energy = {}".format(res.eigenvalues[0] + nre))

Exact ground state energy = -7.882762239225094


In [15]:
[i for i in res.eigenvectors[0] if np.abs(i)>1e-3]

[(-0.7679638858396891-0.6214357598580641j),
 (-0.028828751658577123-0.02332820269682904j),
 (-0.0025891375635314945-0.002095128037709306j),
 (-0.02882875165857784-0.02332820269682099j),
 (0.02531070561116622+0.02048140265973749j),
 (-0.044187917168548606-0.03575682709631196j),
 (0.002923689111462927+0.0023658468816982595j),
 (0.021560465633569618+0.017446711480763724j),
 (-0.002420409940714022-0.0019585937807883157j),
 (-0.002420409940714069-0.0019585937807882433j),
 (0.0013951255181510014+0.0011289344492047158j),
 (0.02156046563356896+0.017446711480763155j),
 (-0.0024204099407140345-0.0019585937807883123j),
 (-0.002420409940713983-0.0019585937807883435j),
 (0.0013951255181510044+0.0011289344492047197j),
 (-0.0025891375635310084-0.0020951280377081473j),
 (-0.04418791716854821-0.03575682709631451j),
 (0.08567707486741709+0.06932982019647185j)]

In [2]:
ha.get_conn([1,0,1,0])

(array([[1., 0., 1., 0.],
        [0., 1., 0., 1.]]),
 array([-0.24507193+0.j,  0.18092562+0.j]))

In [3]:
print(ha.to_sparse())

  (1, 1)	(-1.256399771763353+0j)
  (2, 2)	(-0.47183206191522586+0j)
  (3, 3)	(-1.2445582981540573+0j)
  (4, 4)	(-1.2563997717633526+0j)
  (5, 5)	(-1.8370703030514297+0j)
  (5, 10)	(0.1809256232545573+0j)
  (6, 6)	(-1.0636326748995+0j)
  (6, 9)	(0.1809256232545573+0j)
  (7, 7)	(-1.1606296706630561+0j)
  (8, 8)	(-0.47183206191522586+0j)
  (9, 6)	(0.1809256232545573+0j)
  (9, 9)	(-1.0636326748995+0j)
  (10, 5)	(0.1809256232545573+0j)
  (10, 10)	(-0.24507192913251044+0j)
  (11, 11)	(-0.35319900659226366+0j)
  (12, 12)	(-1.2445582981540573+0j)
  (13, 13)	(-1.160629670663056+0j)
  (14, 14)	(-0.35319900659226366+0j)
  (15, 15)	(0.21440315642325872+0j)


In [4]:
np.random.seed(seed=123)
nk.random.seed(seed=123)

ma = nk.machine.RbmSpin(hi, alpha=2)
ma.init_random_parameters(seed=1234, sigma=0.05)

op = nk.optimizer.Sgd(0.1)

chain_length = 4
sa = nk.sampler.MetropolisExchange(machine=ma, n_chains=chain_length)

n_up = []
tries = 20000
for i in range(tries):
    n_up = []
    sa = nk.sampler.MetropolisExchange(machine=ma, n_chains=chain_length)
    for ss in sa.samples(1):
        for s in ss:
             #print(s, list(s).count(1))
             n_up.append(int(list(s).count(1)))
    if n_up.count(systemData["n_electrons"]) == chain_length: print('found after %d tries' %(i)); break
if i+1==tries:
    print('not found')
else:
    for s in sa.samples(2):
        print(s)

vmc = nk.variational.Vmc(
    hamiltonian=ha,
    sampler=sa,
    optimizer=op,
    n_samples=100000,
    diag_shift=0.1,
    use_iterative=True,
    method='Sr')

found after 89 tries
[[0. 1. 1. 0.]
 [1. 0. 1. 0.]
 [1. 1. 0. 0.]
 [0. 1. 1. 0.]]
[[0. 0. 1. 1.]
 [1. 0. 0. 1.]
 [0. 1. 1. 0.]
 [0. 1. 0. 1.]]


In [8]:
ma.n_par

44

In [6]:
I=[]
E=[]
for it in vmc.iter(5,1):
    print(it,vmc.energy, vmc.energy.mean + nre)
    I.append(it)
    E.append(vmc.energy.mean.real)

0 (-1.00795 + 0.00000i) ± 0.00091 [var=0.24160, R=0.99999] (-0.28788154817764744+1.549888986781332e-06j)
1 (-1.0400 - 0.0000i) ± 0.0016 [var=0.2452, R=1.00000] (-0.3199552902930701-3.708654106069396e-05j)
2 (-1.0759 - 0.0000i) ± 0.0013 [var=0.2469, R=0.99999] (-0.35582961058358176-3.2834114649470694e-05j)
3 (-1.11219 + 0.00001i) ± 0.00093 [var=0.24586, R=0.99999] (-0.392123800377996+9.310681020858973e-06j)
4 (-1.14700 + 0.00000i) ± 0.00088 [var=0.24625, R=0.99999] (-0.42693222944872244+5.262856047489654e-07j)
